In [1]:
!pip install torchvision==0.16.0 torchaudio==2.1.0 torchdata==0.7.0 torchtext==0.16.0

import torch
import torchvision

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print("pytorch", torch.__version__)
print("torchvision", torchvision.__version__)


pytorch 2.1.0+cu121
torchvision 0.16.0+cu121


In [2]:
import numpy as np
from torchvision import tv_tensors
from torchvision.datasets.vision import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive, verify_str_arg
from torchvision.ops import box_area

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import collections
import os
from xml.etree.ElementTree import Element as ET_Element

try:
    from defusedxml.ElementTree import parse as ET_parse
except ImportError:
    from xml.etree.ElementTree import parse as ET_parse
from typing import Any, Callable, Dict, List, Optional, Tuple


class FracAtlasDetection(VisionDataset):
    """FracAtlas dataset."""

    def __init__(
        self,
        root: str,
        image_set: str = "train",
        # download: bool = False,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ):
        super().__init__(root, transforms, transform, target_transform)

        valid_image_sets = ["train", "test"]
        self.image_set = verify_str_arg(image_set, "image_set", valid_image_sets)

        self.url = "https://figshare.com/ndownloader/files/41725659"
        self.filename = "fracatlas.zip"

        # if download:
        if not os.path.isdir("data/FracAtlas"):
            os.makedirs("data", exist_ok=True)
            download_and_extract_archive(self.url, os.path.dirname(self.root), filename=self.filename, remove_finished=True)
            for subdir in ["Fractured", "Non_fractured"]:
              dirpath = os.path.join(self.root, "images")
              subdirpath = os.path.join(dirpath, subdir)
              for f in os.listdir(subdirpath):
                if not f.lower().endswith(".jpg"): continue
                os.rename(os.path.join(subdirpath, f), os.path.join(dirpath, f))
              os.rmdir(subdirpath)
        if not os.path.isdir(self.root):
            raise RuntimeError(
                "Dataset not found or corrupted. You can use download=True to download it"
            )

        image_dir = os.path.join(root, "images")
        target_dir = os.path.join(root, "Annotations", "PASCAL VOC")
        all_images = [os.path.splitext(x)[0] for x in os.listdir(image_dir)]

        # remove images with less than one fracture
        all_images = [x for x in all_images if len(self.parse_voc_xml(ET_parse(os.path.join(target_dir, x + ".xml")).getroot())["annotation"]["object"]) > 0]

        print(len(all_images))

        # 90% of images in train, in which 80% are actually training and 20% validation, and the last 10% in test
        file_names = []
        if image_set == "train":
            file_names = all_images[: int(0.9 * len(all_images))]
        elif image_set == "test":
            file_names = all_images[int(0.9 * len(all_images)) :]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.targets = [os.path.join(target_dir, x + ".xml") for x in file_names]

        assert len(self.images) == len(self.targets)

    @property
    def annotations(self) -> List[str]:
        return self.targets

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert("RGB")
        img = tv_tensors.Image(img)
        voc_dict = self.parse_voc_xml(ET_parse(self.annotations[index]).getroot())
        target = self.voc_dict_to_target(index, voc_dict, img)
        # print(target)

        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET_Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(FracAtlasDetection.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == "annotation":
                def_dic["object"] = [def_dic["object"]]
            voc_dict = {
                node.tag: {
                    ind: v[0] if len(v) == 1 else v for ind, v in def_dic.items()
                }
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

    def voc_dict_to_target(self, index: int, item, img):
        width = int(item["annotation"]["size"]["width"])
        height = int(item["annotation"]["size"]["height"])

        num_fractures = len(item["annotation"]["object"])

        boxes = [
            (
                int(obj["bndbox"]["xmin"]),
                int(obj["bndbox"]["ymin"]),
                int(obj["bndbox"]["xmax"]),
                int(obj["bndbox"]["ymax"]),
            )
            for obj in item["annotation"]["object"]
        ]

        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)

        # extract labels
        labels = torch.ones((num_fractures,), dtype=torch.int64)

        image_id = index
        area = box_area(boxes)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_fractures,), dtype=torch.int64)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="xyxy", canvas_size=img.shape[-2:])
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return target


In [3]:
# import shutil
# shutil.rmtree("data")
# os.mkdir("data")


In [4]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    # transforms.append(T.ToImage())
    # if train:
    #     # transforms.append(T.RandomPhotometricDistort())
    #     # transforms.append(T.RandomZoomOut(fill={tv_tensors.Image: (0,0,0), "others": 0}))
    #     transforms.append(T.RandomHorizontalFlip(0.5))
    # transforms.append(T.ClampBoundingBoxes())
    # transforms.append(T.SanitizeBoundingBoxes())
    transforms.append(T.ToDtype(dtype=torch.float, scale=True))
    return T.Compose(transforms)


In [5]:
# os.makedirs("lib", exist_ok=True)
if not os.path.exists("engine.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py -O engine.py"
    )
if not os.path.exists("utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py -O utils.py"
    )
if not os.path.exists("transforms.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py -O transforms.py"
    )
if not os.path.exists("coco_utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py -O coco_utils.py"
    )
if not os.path.exists("coco_eval.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py -O coco_eval.py"
    )

# you also need to tweak them a bit to work with relative imports


In [6]:
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

from utils import collate_fn

model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)
dataset = FracAtlasDetection("data/FracAtlas", image_set="train", transform=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
  dataset,
  batch_size=2,
  shuffle=True,
  # num_workers=4,
  collate_fn=collate_fn,
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])


Downloading: "https://download.pytorch.org/models/ssdlite320_mobilenet_v3_large_coco-a79551df.pth" to /root/.cache/torch/hub/checkpoints/ssdlite320_mobilenet_v3_large_coco-a79551df.pth
100%|██████████| 13.4M/13.4M [00:00<00:00, 75.9MB/s]


100%|██████████| 338412751/338412751 [00:16<00:00, 20040141.32it/s]


Extracting data/fracatlas.zip to data
717
{'bbox_regression': tensor(2.8883, grad_fn=<DivBackward0>), 'classification': tensor(4.6364, grad_fn=<DivBackward0>)}
{'boxes': tensor([[ 10.5422,   5.2503, 394.1647, 296.1993],
        [ 12.3853,   3.2641, 392.1215, 297.6234],
        [ 10.4345,   8.8259, 390.6984, 294.7046],
        ...,
        [246.9263, 225.7427, 257.6853, 241.2783],
        [367.2489, 262.0085, 378.5447, 273.0168],
        [362.7723, 167.2736, 378.4266, 179.6777]], grad_fn=<StackBackward0>), 'scores': tensor([0.0667, 0.0586, 0.0335, 0.0286, 0.0286, 0.0268, 0.0267, 0.0258, 0.0258,
        0.0252, 0.0252, 0.0251, 0.0249, 0.0244, 0.0241, 0.0237, 0.0230, 0.0228,
        0.0226, 0.0225, 0.0224, 0.0224, 0.0221, 0.0221, 0.0221, 0.0221, 0.0218,
        0.0218, 0.0218, 0.0217, 0.0216, 0.0215, 0.0214, 0.0214, 0.0213, 0.0212,
        0.0210, 0.0210, 0.0210, 0.0208, 0.0208, 0.0207, 0.0207, 0.0207, 0.0207,
        0.0207, 0.0206, 0.0205, 0.0204, 0.0203, 0.0203, 0.0201, 0.0201, 0.0201,

In [7]:
from engine import train_one_epoch, evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)

# use our dataset and defined transformations
dataset = FracAtlasDetection('data/FracAtlas', image_set="train", transform=get_transform(train=True))
dataset_test = FracAtlasDetection('data/FracAtlas', image_set="test", transform=get_transform(train=False))

# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-80])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-20:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=3,
    shuffle=True,
    # num_workers=4,
    collate_fn=collate_fn,
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    # num_workers=4,
    collate_fn=collate_fn,
)

model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
# lr_scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=3,
#     gamma=0.1
# )

# let's train it for 5 epochs
num_epochs = 30

for epoch in range(num_epochs):
    model.eval()
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    # lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

print("Saving")
torch.save(model, 'fracatlas_ssdlite-mobilenetv3.pth')
print("Saved")


device: cuda
717
717
Epoch: [0]  [  0/215]  eta: 0:11:13  lr: 0.000028  loss: 13.5866 (13.5866)  bbox_regression: 5.6834 (5.6834)  classification: 7.9033 (7.9033)  time: 3.1332  data: 0.1538  max mem: 513
Epoch: [0]  [ 10/215]  eta: 0:01:22  lr: 0.000262  loss: 11.9775 (12.2104)  bbox_regression: 5.6834 (5.3359)  classification: 7.2006 (6.8746)  time: 0.4044  data: 0.0707  max mem: 528
Epoch: [0]  [ 20/215]  eta: 0:00:54  lr: 0.000495  loss: 11.1548 (10.9059)  bbox_regression: 5.6023 (5.5788)  classification: 4.2607 (5.3272)  time: 0.1366  data: 0.0698  max mem: 528
Epoch: [0]  [ 30/215]  eta: 0:00:43  lr: 0.000729  loss: 7.9456 (9.7223)  bbox_regression: 4.9734 (5.0111)  classification: 3.3578 (4.7112)  time: 0.1458  data: 0.0790  max mem: 528
Epoch: [0]  [ 40/215]  eta: 0:00:40  lr: 0.000962  loss: 6.9208 (9.4653)  bbox_regression: 3.8118 (5.1046)  classification: 3.2802 (4.3607)  time: 0.1829  data: 0.1079  max mem: 601
Epoch: [0]  [ 50/215]  eta: 0:00:34  lr: 0.001195  loss: 8.6683